# 감정선 classification

감정선

In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from PIL import Image
from tqdm.auto import tqdm

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd /content/drive/MyDrive/Colab Notebooks/AI_Project/인지프 팀플/classification/data

/content/drive/MyDrive/Colab Notebooks/AI_Project/인지프 팀플/classification/data


In [4]:
df = pd.read_excel('label4.xlsx')
df = df[~df['label'].str.contains("-")]  # Filter out invalid entries

In [5]:
print(df)

     File Number  life head  heart label
0              1     0    0      0   000
1              2     0    0      0   000
2              5     0    1      0   010
3             12     0    0      1   001
4             13     0    1      0   010
..           ...   ...  ...    ...   ...
976         5353     0    0      1   001
977         5354     1    0      1   101
978         5366     0    0      0   000
979         5367     0    0      0   000
981         5385     0    0      1   001

[927 rows x 5 columns]


In [6]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [7]:
def load_and_preprocess_images(image_df):
    images = []
    for idx, row in tqdm(image_df.iterrows(), total=image_df.shape[0]):
        image_path = os.path.join('./heart', f"image{row['File Number']}.png")
        image = Image.open(image_path).resize((128, 128))
        image = np.array(image) / 255.0
        if image.ndim == 2 or image.shape[2] == 1:
            image = np.stack((image,)*3, axis=-1)  # Convert grayscale to RGB
        images.append(image)
    return np.array(images)

In [8]:
train_images = load_and_preprocess_images(train_df)
test_images = load_and_preprocess_images(test_df)

  0%|          | 0/741 [00:00<?, ?it/s]

  0%|          | 0/186 [00:00<?, ?it/s]

In [9]:
train_labels = to_categorical(train_df['heart'], num_classes=2)
test_labels = to_categorical(test_df['heart'], num_classes=2)

In [10]:
print(train_labels)

[[1. 0.]
 [0. 1.]
 [1. 0.]
 ...
 [1. 0.]
 [0. 1.]
 [0. 1.]]


In [11]:
train_labels = train_df['heart'].values
test_labels = test_df['heart'].values

In [12]:
print(train_labels)

[0 1 0 1 0 1 1 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 1 1 1 0 0 1 0 0 0 1 0
 1 1 1 1 0 1 1 0 1 1 1 1 0 1 0 1 0 0 0 1 1 1 0 1 0 0 1 0 1 0 1 0 1 0 0 1 0
 0 0 1 1 0 1 0 1 1 1 0 1 0 0 0 1 1 0 0 0 0 1 0 1 0 1 0 1 1 1 1 1 1 1 0 0 0
 0 0 0 0 0 0 0 1 0 1 1 0 1 0 0 1 0 1 0 0 1 1 1 0 0 0 1 0 0 1 0 0 1 0 1 1 1
 0 0 0 1 1 0 0 1 0 1 0 0 0 0 1 0 1 0 0 0 1 1 0 0 0 1 1 1 1 0 1 0 1 1 1 0 1
 0 1 1 0 0 1 0 0 1 0 0 0 1 1 1 1 0 1 1 1 0 0 1 1 0 1 0 1 0 0 0 1 0 1 0 0 0
 1 1 1 0 1 1 0 1 1 1 0 1 0 1 0 1 1 1 1 1 0 1 0 0 0 1 1 1 0 0 0 0 1 0 0 0 0
 0 0 0 1 0 1 0 0 0 1 0 1 0 1 1 0 0 1 0 1 1 1 0 1 1 1 1 1 0 1 0 1 0 1 1 0 1
 0 1 1 0 0 1 0 0 0 1 0 0 0 0 1 0 1 1 0 0 0 0 0 0 1 0 1 1 0 1 1 1 0 0 1 0 1
 0 0 1 1 0 1 1 1 0 1 0 1 1 1 0 0 1 0 0 1 0 1 1 0 1 0 1 1 1 0 1 1 0 0 0 0 0
 0 0 0 1 1 1 0 0 0 0 0 1 0 1 0 1 1 1 0 0 1 1 1 1 1 1 0 1 0 0 0 1 0 1 0 1 0
 1 0 0 0 1 1 1 0 0 1 0 1 0 1 1 0 1 0 0 0 0 1 0 1 1 1 0 1 1 1 1 0 0 1 0 0 1
 1 0 0 1 0 1 0 1 0 0 0 1 0 1 0 0 0 0 1 1 1 1 0 1 0 1 1 1 0 0 0 0 1 0 0 0 0
 0 1 1 1 1 1 0 0 0 0 0 0 

In [13]:
# Initialize MobileNetV2 model with pretrained ImageNet weights
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
base_model.trainable = False  # Freeze base model


model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(8, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # 1 unit for binary classification
])

In [14]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])

In [15]:
# Callbacks for early stopping and saving the best model
callbacks = [
    EarlyStopping(patience=5, verbose=1),
    ModelCheckpoint('best_model.h5', save_best_only=True, save_weights_only=True)
]

history = model.fit(
    train_images, train_labels,
    validation_data=(test_images, test_labels),
    batch_size=32,
    epochs=50,  # May increase if model is not overfitting
    callbacks=callbacks
)

# Load the best weights and unfreeze the base_model for fine-tuning
model.load_weights('best_model.h5')
base_model.trainable = True

Epoch 1/50
24/24 [==============================] - 15s 450ms/step - loss: 0.7037 - binary_accuracy: 0.5587 - val_loss: 0.6870 - val_binary_accuracy: 0.6290
Epoch 2/50
24/24 [==============================] - 10s 436ms/step - loss: 0.6850 - binary_accuracy: 0.5843 - val_loss: 0.6792 - val_binary_accuracy: 0.6398
Epoch 3/50
24/24 [==============================] - 11s 447ms/step - loss: 0.6744 - binary_accuracy: 0.5938 - val_loss: 0.6373 - val_binary_accuracy: 0.7581
Epoch 4/50
24/24 [==============================] - 10s 396ms/step - loss: 0.6802 - binary_accuracy: 0.5992 - val_loss: 0.6486 - val_binary_accuracy: 0.7043
Epoch 5/50
24/24 [==============================] - 14s 579ms/step - loss: 0.6448 - binary_accuracy: 0.6775 - val_loss: 0.5981 - val_binary_accuracy: 0.7151
Epoch 6/50
24/24 [==============================] - 11s 460ms/step - loss: 0.6307 - binary_accuracy: 0.6356 - val_loss: 0.5917 - val_binary_accuracy: 0.7527
Epoch 7/50
24/24 [==============================] - 9s 359

In [16]:
# Evaluate the model on the test set.
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")

6/6 [==============================] - 2s 254ms/step - loss: 0.3667 - binary_accuracy: 0.8226
Test accuracy: 82.26%


In [17]:
from sklearn.metrics import f1_score

# Predict on the test set
test_predictions = model.predict(test_images)
test_predictions = (test_predictions > 0.5).astype("int32")  # Convert probabilities to binary predictions

# Calculate the F1 score
f1 = f1_score(test_labels, test_predictions)

print(f"F1 Score: {f1}")

6/6 [==============================] - 2s 247ms/step
F1 Score: 0.7924528301886793


In [18]:
# Recompile the model with a low learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),  # Low learning rate for fine-tuning
              loss='binary_crossentropy',
              metrics=['binary_accuracy'])

In [19]:
# Fine-tune the model
history_fine = model.fit(
    train_images, train_labels,
    validation_data=(test_images, test_labels),
    batch_size=32,
    epochs=20,
    callbacks=callbacks
)

Epoch 1/20
24/24 [==============================] - 59s 2s/step - loss: 3.8595 - binary_accuracy: 0.4696 - val_loss: 0.3939 - val_binary_accuracy: 0.7957
Epoch 2/20
24/24 [==============================] - 38s 2s/step - loss: 3.3184 - binary_accuracy: 0.4818 - val_loss: 0.4705 - val_binary_accuracy: 0.7581
Epoch 3/20
24/24 [==============================] - 44s 2s/step - loss: 2.9098 - binary_accuracy: 0.4845 - val_loss: 0.5964 - val_binary_accuracy: 0.7366
Epoch 4/20
24/24 [==============================] - 37s 2s/step - loss: 2.3917 - binary_accuracy: 0.5196 - val_loss: 0.8247 - val_binary_accuracy: 0.6667
Epoch 5/20
24/24 [==============================] - 38s 2s/step - loss: 2.1515 - binary_accuracy: 0.5115 - val_loss: 1.2910 - val_binary_accuracy: 0.5753
Epoch 6/20
24/24 [==============================] - 39s 2s/step - loss: 1.6455 - binary_accuracy: 0.5452 - val_loss: 1.6268 - val_binary_accuracy: 0.5323
Epoch 6: early stopping


In [20]:
# Evaluate the model on the test set.
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")

6/6 [==============================] - 2s 251ms/step - loss: 1.6268 - binary_accuracy: 0.5323
Test accuracy: 53.23%


In [21]:
from sklearn.metrics import f1_score

# Predict on the test set
test_predictions = model.predict(test_images)
test_predictions = (test_predictions > 0.5).astype("int32")  # Convert probabilities to binary predictions

# Calculate the F1 score
f1 = f1_score(test_labels, test_predictions)

print(f"F1 Score: {f1}")

6/6 [==============================] - 3s 361ms/step
F1 Score: 0.6506024096385542
